In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

___
## 시계열 데이터 분석 프레임워크 : ETS (오류, 추세, 계절성) 모델
___
	1. 오류 (Error): 시계열 데이터의 예측 값과 실제 값 간의 차이. 이는 모델이 데이터의 변동성을 얼마나 잘 설명하는지를 나타냄.
   
	2. 추세 (Trend): 데이터가 장기적으로 증가하거나 감소하는 패턴을 의미. 추세는 선형적일 수도 있고, 비선형적일 수도 있음.
   
	3. 계절성 (Seasonal): 데이터가 일정 주기로 반복되는 패턴을 의미. 계절성은 주기적 변동성을 나타내며, 월별, 분기별, 연간 등 다양한 주기로 나타날 수 있음.


## ETS

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

In [3]:
# 데이터 로드
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url, index_col='Month', parse_dates=True)

In [9]:
# 데이터 시각화
data.iplot(title='AirPassengers Data')

In [16]:
# ETS 분해 도구 적용
result = seasonal_decompose(data['Passengers'], model='multiplicative')


In [17]:
# 예측 값과 각 구성 요소 분리
observed, resid, sesonal, trend = [ getattr(result, attr) for attr in ['observed', 'resid', 'seasonal', 'trend']]

In [19]:
# 구성 요소 시각화
# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=("Observed", "Trend", "Seasonal", "Residual"))

# Observed
fig.add_trace(go.Scatter(x=observed.index, y=observed, mode='lines', name='Observed'), row=1, col=1)

# Trend
fig.add_trace(go.Scatter(x=trend.index, y=trend, mode='lines', name='Trend'), row=1, col=2)

# Seasonal
fig.add_trace(go.Scatter(x=sesonal.index, y=sesonal, mode='lines', name='Seasonal'), row=2, col=1)

# Residual
fig.add_trace(go.Scatter(x=resid.index, y=resid, mode='lines', name='Residual'), row=2, col=2)

# Update layout
fig.update_layout(title_text="시계열 데이터의 구성 요소 분해", showlegend=False)

# Show plot
fig.show()

| 구분     | 설명                                                                 |
|----------|----------------------------------------------------------------------|
| Trend    | 데이터의 장기적인 방향. 시간에 따라 증가하거나 감소하는 경향. |
| Seasonal | 데이터의 주기적인 변동. 계절이나 월별, 주별 등 일정한 주기로 반복되는 패턴. |
| Residual | 데이터에서 Trend와 Seasonal을 제거한 후 남은 부분. 주로 불규칙한 변동이나 잡음을 포함. |


___
## SMA(단순 이동 평균)과 EWMA(지수 가중 이동 평균)
___

Simple Moving Average (SMA)

In [21]:
data['6M-SMA'] = data['Passengers'].rolling(window=6).mean()
data['12M-SMA'] = data['Passengers'].rolling(window=12).mean()

In [23]:
data.head(15)

,Passengers,6M-SMA,12M-SMA
Month,,,
1949-01-01,112,NaN,NaN
1949-02-01,118,NaN,NaN
1949-03-01,132,NaN,NaN
1949-04-01,129,NaN,NaN
1949-05-01,121,NaN,NaN
1949-06-01,135,124.500000,NaN
1949-07-01,148,130.500000,NaN
1949-08-01,148,135.500000,NaN
1949-09-01,136,136.166667,NaN


In [24]:
px.line(data).update_layout(legend=dict(x=0, y=1), title='SMA(Simple Moving Average)')

Exponential Weighted Moving Average (EWMA)

* SMA의 한계
  
   * 윈도우 크기가 작으면 잡음이 더 커짐.
  
   * 윈도우 크기만큼 항상 지연이 발생.
   * 평균화로 인해 데이터의 최고점이나 최저점에 도달하지 못하게됨.
   * 미래의 행동을 예측하지 못하고, 데이터의 추세만을 보여줌.
   * 극단적인 과거 값이 단순 이동 평균을 크게 왜곡.

* <font color="red">EWMA(Exponential Weighted Moving Average, 지수 가중 이동 평균)</font>
  
    * EWMA는 SMA의 지연 효과를 줄이고 최근에 발생한 값에 더 많은 가중치를 부여
  
    * 최근 값에 부여되는 가중치의 양은 EWMA에 사용된 실제 매개변수와 윈도우 크기에 따라 달라짐.
    * [EWMA의 수학적 배경에 대한 Pandas 문서](http://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#exponentially-weighted-windows).


* 공식

  $EWMA_t = \alpha \cdot x_t + (1 - \alpha) \cdot EWMA_{t-1}$


	* ( EWMA_t )는 시간 ( t )에서의 지수 가중 이동 평균 값
	* ( x_t )는 시간  t 에서의 실제 데이터 값
  
*  $\alpha$(가중치 감소율)  계산
  	$\alpha = \frac{2}{span + 1} $

	* span은 평활 정도를 조정하는 매개변수

	1.	adjust=True:
		* 이 경우 초기값부터 모든 데이터 포인트에 대해 조정된 평균을 계산
		* 전체 데이터에 대한 가중 평균을 계산. 초기 시점의 데이터는 이후의 데이터에 덜 영향을 미침
		* 공식:
 			$EWMA_t = \frac{\sum_{i=0}^{t} \left( \alpha (1 - \alpha)^i x_{t-i} \right)}{\sum_{i=0}^{t} \left( \alpha (1 - \alpha)^i \right)} $

	2.	adjust=False:
		* 이 경우 초기값에서부터 점진적으로 이동 평균을 계산
		* 현재 시점에서 과거 데이터 포인트에 대한 가중치가 계속 줄어들어 직접적으로 계산됨
		* 공식:
			$EWMA_t = \alpha \cdot x_t + (1 - \alpha) \cdot EWMA_{t-1} $



In [29]:
data['EWMA_adjust_True']

Month
1949-01-01    112.000000
1949-02-01    115.250000
1949-03-01    121.787529
1949-04-01    124.064224
1949-05-01    123.231685
                 ...    
1960-08-01    494.898620
1960-09-01    496.914217
1960-10-01    491.388952
1960-11-01    475.790652
1960-12-01    469.053629
Name: EWMA_adjust_True, Length: 144, dtype: float64

In [26]:
span = 12
data['EWMA_adjust_True'] = data['Passengers'].ewm(span=span, adjust=True).mean()

# span 값을 12로 설정하여 지수 가중 이동 평균 계산 (adjust=False)
data['EWMA_adjust_False'] = data['Passengers'].ewm(span=span, adjust=False).mean()

In [30]:
px.line(data[['Passengers', 'EWMA_adjust_True', 'EWMA_adjust_False']]).update_layout(legend=dict(x=0, y=1), title='EWMA(Exponential Weighted Moving Average)')
# adjust=True: 모든 데이터 포인트에 대해 조정된 평균을 계산하므로 초기값부 전체 데이터에 대한 영향을 받음
# adjust=False: 초기값에서 시작하여 점진적으로 평균을 계산하므로, 초기 시점에서의 변화가 상대적으로 더 크게 반영됨

___
## Hole-Winters Method (이중/삼중 지수평활법)
___

* 시계열 데이터를 분석하고 <font color="red">예측하는 데 사용되는 기법</font>

* 특히 추세와 계절성이 있는 데이터에 유용
* 세 가지 주요 요소(수준, 추세, 계절성)를 결합하여 시계열 데이터를 모델링
* 가법적(Additive) 모델과 승법적(Multiplicative) 모델 같은 변형 모델 제공

* 프레임워크
    1.	수준(Level, ($L_t$)): 현재 시간 ($t$)에서의 데이터의 기본값 또는 평균 수준
    2.	추세(Trend, ($T_t$)): 시간의 경과에 따라 데이터가 증가하거나 감소하는 패턴
    3.	계절성(Seasonal, ($S_t$)): 일정 주기로 반복되는 패턴

* <font color="red">단순 지수평활법</font>
  
    - 하나의 평활계수 $\alpha$를 사용함.
    - 데이터에 추세가 있을 때 예측을 잘하지 못함.
    - 공식
        $y_0 = x_0$

        $y_t = (1 - \alpha) y_{t-1} + \alpha x_t$



* <font color="red">가법적 모델 (Additive Model)</font>

  - 데이터의 변동폭이 일정할 때 사용

  - **수준(Level, $L_t$)**
  
    $L_t = \alpha (Y_t - S_{t-m}) + (1 - \alpha) (L_{t-1} + T_{t-1})$
  
  - **추세(Trend, $T_t$)**
  
    $T_t = \beta (L_t - L_{t-1}) + (1 - \beta) T_{t-1}$
  
  - **계절성(Seasonal, $S_t$)**
  
    $S_t = \gamma (Y_t - L_t) + (1 - \gamma) S_{t-m}$
    


  - $Y_t$: 시간 $t$에서의 실제 데이터 값
  - $\alpha$, $\beta$, $\gamma$: 각각 수준, 추세, 계절성에 대한 평활 계수(0과 1 사이의 값)
  - $m$: 계절성 주기(예: 월별 데이터의 경우 12)


  * <font color="red">승법적 모델 (Multiplicative Model)</font>

    * 데이터의 변동폭이 시계열의 수준에 따라 달라질 때 사용

    - **수준(Level, $L_t$)**:
      
      $L_t = \alpha \left( \frac{Y_t}{S_{t-m}} \right) + (1 - \alpha) (L_{t-1} + T_{t-1})$
    
    - **추세(Trend, $T_t$)**:
     
      $T_t = \beta (L_t - L_{t-1}) + (1 - \beta) T_{t-1}$
    
    - **계절성(Seasonal, $S_t$)**:
     
      $S_t = \gamma \left( \frac{Y_t}{L_t} \right) + (1 - \gamma) S_{t-m}$



  * <font color="red">예측(Forecasting)</font>

    - **가법적 모델**:
      $\hat{Y}_{t+h} = L_t + hT_t + S_{t-m+h}$

    - **승법적 모델**:
      $\hat{Y}_{t+h} = (L_t + hT_t) \cdot S_{t-m+h}$

    여기서 $h$는 예측하려는 미래 시점의 길이


* 매개변수 조정법

  - 시계열이 직선 경사 추세를 보이는 경우, trend=additive(추가) 조정을 사용함.

  - 시계열이 지수(곡선) 추세를 보이는 경우, trend=multiplicative(곱셈) 조정을 사용함.
  - 시계열이 주기적으로 반복되는 경우, seasonal=additive(추가) 조정을 사용함.
  - 시계열이 주기적으로 반복되면서 변동폭이 일정하지 않은 경우, seasonal=multiplicative(곱셈) 조정을 사용함.
  - 매년 반복되는 패턴을 보이는 월별 데이터를 살펴보는 경우 seasonal_periods=12를 사용함.
  - 일반적으로 $\alpha$, $\beta$ 및 $\gamma$의 값이 높을수록 (1에 가까운 값) 최근 데이터에 더 많은 비중을 둠.


In [34]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

단순 지수 평활법

In [32]:
span = 12
alpha = 2 / (span + 1)

In [36]:
# optimized=False를 로 적합하면 값이 한 행 아래로 이동하게 됨.
# 이를 수정하기 위해 .fittedvalues 뒤에 .shift(-1)를 추가
data['ewma12'] = data['Passengers'].ewm(alpha=alpha, adjust=False).mean()
data['ses12'] = SimpleExpSmoothing(data['Passengers']).fit(smoothing_level=alpha, optimized=False).fittedvalues.shift(-1)
data.head()

,Passengers,6M-SMA,12M-SMA,EWMA_adjust_True,EWMA_adjust_False,ewma12,ses12
Month,,,,,,,
1949-01-01,112,NaN,NaN,112.000000,112.000000,112.000000,112.000000
1949-02-01,118,NaN,NaN,115.250000,112.923077,112.923077,112.923077
1949-03-01,132,NaN,NaN,121.787529,115.857988,115.857988,115.857988
1949-04-01,129,NaN,NaN,124.064224,117.879836,117.879836,117.879836
1949-05-01,121,NaN,NaN,123.231685,118.359861,118.359861,118.359861


이중 지수 평활법


추세관련 beta 평활계수 추가

$l_t = (1 - \alpha) l_{t-1} + \alpha x_t, \quad \text{level} \\$
$b_t = (1-\beta)b_{t-1} + \beta(l_t-l_{t-1}), \quad \text{trend} \\$
$y_t = l_t + b_t, \quad \text{fitted model} \\$
$\hat y_{t+h} = l_t + hb_t, \quad \text{forecasting model (h = 미래의 기간 수)}$

In [38]:
data['2es_add_12'] = ExponentialSmoothing(data['Passengers'], trend='add').fit().fittedvalues.shift(-1) # 2중지수 평활 가법적 모델
data['2es_mul_12'] = ExponentialSmoothing(data['Passengers'], trend='mul').fit().fittedvalues.shift(-1) # 2중지수 평활 승법적 모델
data[['Passengers', '2es_add_12', '2es_mul_12']].iplot()


삼중 지수 평활법 (Holt-Winters Method)


계절성 관련 gamma 평활계수 추가

$l_t = (1 - \alpha) l_{t-1} + \alpha x_t, \quad \text{level} \\$
$b_t = (1-\beta)b_{t-1} + \beta(l_t-l_{t-1}), \quad \text{trend} \\$
$s_t = (1-\gamma)s_{t-1} + \gamma(y_t-l_t), \quad \text{seasonal} \\$
$y_t = l_t + b_t + s_t, \quad \text{fitted model} \\$
$\hat y_{t+h} = l_t + hb_t + s_{t-m+h}, \quad \text{forecasting model (h = 미래의 기간 수)}$



In [39]:
data['3es_add_12'] = ExponentialSmoothing(data['Passengers'],trend='add',seasonal='add',seasonal_periods=12).fit().fittedvalues # 3중지수 평활 가법적 모델
data['3es_mul_12'] = ExponentialSmoothing(data['Passengers'],trend='mul',seasonal='mul',seasonal_periods=12).fit().fittedvalues # 3중지수 평활 승법적 모델
data[['Passengers', '3es_add_12', '3es_mul_12']].iplot()

In [43]:
from statsmodels.tools.eval_measures import rmse
import numpy as np


# 실제 값과 예측 값
actual = data['Passengers']
predicted_ses = data['ses12'].dropna()
predicted_2es_add = data['2es_add_12'].dropna()
predicted_2es_mul = data['2es_mul_12'].dropna()
predicted_3es_add = data['3es_add_12'].dropna()
predicted_3es_mul = data['3es_mul_12'].dropna()

# RMSE 계산
rmse_ses = rmse(actual.loc[predicted_ses.index], predicted_ses)
rmse_2es_add = rmse(actual.loc[predicted_2es_add.index], predicted_2es_add)
rmse_2es_mul = rmse(actual.loc[predicted_2es_mul.index], predicted_2es_mul)
rmse_3es_add = rmse(actual.loc[predicted_3es_add.index], predicted_3es_add)
rmse_3es_mul = rmse(actual.loc[predicted_3es_mul.index], predicted_3es_mul)

# RMSE 출력
print(f"단순 지수 평활법 RMSE: {rmse_ses}")
print(f"2중지수 평활 가법적 모델 RMSE: {rmse_2es_add}")
print(f"2중지수 평활 승법적 모델 RMSE: {rmse_2es_mul}")
print(f"3중지수 평활 가법적 모델 RMSE: {rmse_3es_add}")
print(f"3중지수 평활 승법적 모델 RMSE: {rmse_3es_mul}")


단순 지수 평활법 RMSE: 41.42319911747645
2중지수 평활 가법적 모델 RMSE: 1.4883334033683662
2중지수 평활 승법적 모델 RMSE: 5.290048214333465
3중지수 평활 가법적 모델 RMSE: 12.237335271937923
3중지수 평활 승법적 모델 RMSE: 10.476593312120297


#### 결론

| 방법 | 장점 | 단점 |
| --- | --- | --- |
| 단순 이동 평균 (SMA) | 계산이 간단하고 이해하기 쉬움 | 최근 데이터에 대한 가중치가 동일하여 최신 정보 반영이 느림 |
| 지수 이동 평균 (EWMA) | 최근 데이터에 더 큰 가중치를 부여하여 최신 정보 반영이 빠름 | 알파 값 선택에 따라 결과가 크게 달라질 수 있음 |
| 단순 지수 평활법 (SES) | 단기 예측에 유용하며 계산이 간단함 | 추세나 계절성을 반영하지 못함 |
| 이중 지수 평활법 (Holt's Linear Trend Model) | 추세를 반영하여 예측 정확도를 높임 | 계절성을 반영하지 못함 |
| 삼중 지수 평활법 (Holt-Winters Method) | 추세와 계절성을 모두 반영하여 예측 정확도가 높음 | 모델이 복잡하고, 알파, 베타, 감마 값 선택이 어려움 |
